In [1]:
import pandas_ta as ta
from quantnb.lib import find_files, pd
import talib
import time
import numpy as np
from prettytable import PrettyTable


datas = {}
assets = find_files("./data", "binance")

for asset in assets:
    datas[asset.split("-")[1]] = pd.read_parquet(asset)


keys = list(datas.keys())

data = datas[keys[0]]
print(data)

./data
                        open  ...       volume
Date                          ...             
2018-05-04 08:00:00  0.50000  ...   5090857.33
2018-05-04 09:00:00  0.92300  ...   3326696.10
2018-05-04 10:00:00  0.90199  ...   1176810.48
2018-05-04 11:00:00  0.90128  ...   1714233.92
2018-05-04 12:00:00  0.91596  ...   1323800.17
...                      ...  ...          ...
2023-08-02 07:00:00  0.69350  ...  10121095.00
2023-08-02 08:00:00  0.69480  ...  10320024.00
2023-08-02 09:00:00  0.69470  ...   5349201.00
2023-08-02 10:00:00  0.69420  ...   5752090.00
2023-08-02 11:00:00  0.69740  ...   1221871.00

[45900 rows x 5 columns]


In [2]:
# correlation_matrix = ta_sma.corr(talib_sma)
# print(correlation_matrix)
def get_time(library, function, data=None, **args):
    start = time.time()
    result = getattr(library, function)(data, **args)
    end = time.time()
    ta_time = end - start
    return np.round(ta_time, 10)


t = PrettyTable(["Method", "Pandas_TA(s)", "Talib(s)", "Improvement"])

# ================================================================================================ #
#                                               SMA                                                #
# ================================================================================================ #
pandas_time = get_time(ta, "sma", data=data["close"], length=20)
talib_time = get_time(talib, "SMA", data=data["close"], timeperiod=20)
improvement = f"{np.round((pandas_time - talib_time) / pandas_time * 100, 2)}%"
t.add_row(["SMA", pandas_time, talib_time, improvement])

# ================================================================================================ #
#                                               EMA                                                #
# ================================================================================================ #
pandas_time = get_time(ta, "ema", data=data["close"], length=20)
talib_time = get_time(talib, "EMA", data=data["close"], timeperiod=20)
improvement = f"{np.round((pandas_time - talib_time) / pandas_time * 100, 2)}%"
t.add_row(["EMA", pandas_time, talib_time, improvement])

# ================================================================================================ #
#                                               ATR                                                #
# ================================================================================================ #
pandas_time = get_time(
    ta, "atr", data=data["high"], low=data["low"], close=data["close"], length=14
)
talib_time = get_time(
    talib,
    "ATR",
    data=data["high"],
    low=data["low"],
    close=data["close"],
    timeperiod=14,
)
improvement = f"{np.round((pandas_time - talib_time) / pandas_time * 100, 2)}%"
t.add_row(["ATR", pandas_time, talib_time, improvement])


# ================================================================================================ #
#                                       Bollinger Bands                                            #
# ================================================================================================ #
pandas_time = get_time(ta, "bbands", data=data["close"], length=20)
talib_time = get_time(talib, "BBANDS", data=data["close"], timeperiod=20)
improvement = f"{np.round((pandas_time - talib_time) / pandas_time * 100, 2)}%"
t.add_row(["Bollinger Bands", pandas_time, talib_time, improvement])


# ================================================================================================ #
#                                         Print OUTPUT                                             #
# ================================================================================================ #
print(t)

+-----------------+--------------+--------------+-------------+
|      Method     | Pandas_TA(s) |   Talib(s)   | Improvement |
+-----------------+--------------+--------------+-------------+
|       SMA       | 0.0004887581 | 0.0001530647 |    68.68%   |
|       EMA       | 0.0002169609 | 0.0001747608 |    19.45%   |
|       ATR       | 0.0007224083 | 0.0003979206 |    44.92%   |
| Bollinger Bands | 0.0020177364 | 0.0003914833 |    80.6%    |
+-----------------+--------------+--------------+-------------+
